# Test Takuan api

In [38]:
!pip install requests

In [39]:
import sys
import os

# Add the project directory to sys.path
sys.path.append(os.path.abspath('.'))


# Generate small test dataset (RCM)
from utils.generate_dataset import main
main()

Simulated raw count matrix saved to 'simulated_raw_counts.csv'.
Differential expression report saved to 'differential_expression_report.csv'.


In [40]:
import requests

BASE_URL = "http://tds:5000"

def send_get_request(endpoint, headers):
    """
    Sends a GET request to a Takuan endpoint with the provided headers.
    """
    url = f"{BASE_URL}{endpoint}"
    try:
        response = requests.get(url, headers=headers)
        print(f"GET {endpoint} - Status Code: {response.status_code}")
        if response.status_code == 200:
            print("Response JSON:")
            print(response.json())
        else:
            print("Error: Unable to fetch data from the endpoint.")
            print(f"Response Content: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the GET request to {endpoint}: {e}")

def send_post_request(endpoint, headers, payload):
    """
    Sends a POST request to a Takuan endpoint with the provided headers and payload.
    """
    url = f"{BASE_URL}{endpoint}"
    try:
        response = requests.post(url, headers=headers, json=payload)
        print(f"POST {endpoint} - Status Code: {response.status_code}")
        if response.status_code in [200, 201]:
            print("Response JSON:")
            print(response.json())
        else:
            print("Error: Unable to post data to the endpoint.")
            print(f"Response Content: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the POST request to {endpoint}: {e}")

def send_post_request_with_file(endpoint, headers, files):
    """
    Sends a POST request with files (multipart/form-data).
    """
    url = f"{BASE_URL}{endpoint}"
    try:
        response = requests.post(url, headers=headers, files=files)
        print(f"POST {endpoint} - Status Code: {response.status_code}")
        if response.status_code in [200, 201]:
            print("Response JSON:")
            print(response.json())
        else:
            print("Error: Unable to post file to the endpoint.")
            print(f"Response Content: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the POST request to {endpoint}: {e}")


In [41]:
# POST to /experiment
# Create an experiment
headers_post = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload_experiment = {
    "experiment_result_id": "try1",
    "assembly_id": "ai1",
    "assembly_name": "an1",
    "extra_properties": {}
}
send_post_request("/experiment", headers_post, payload_experiment)

POST /experiment - Status Code: 500
Error: Unable to post data to the endpoint.
Response Content: Internal Server Error


In [42]:
# POST to 
# Ingest an experiment

headers_ingest = {
    "accept": "application/json"
}

# Open the file using a context manager.
file_path = "simulated_raw_counts.csv" ## previously generated
with open(file_path, "rb") as file_obj:
    files = {
        "rcm_file": (file_path, file_obj, "text/tab-separated-values")
    }
    send_post_request_with_file("/experiment/try1/ingest", headers_ingest, files)



POST /experiment/try1/ingest - Status Code: 500
Error: Unable to post file to the endpoint.
Response Content: Internal Server Error


In [43]:
# GET from /experiment
headers_get = {
    "accept": "application/json"
}
send_get_request("/experiment", headers_get)


GET /experiment - Status Code: 200
Response JSON:
[{'experiment_result_id': 'try1', 'assembly_id': 'ai1', 'assembly_name': 'an1', 'extra_properties': {}}, {'experiment_result_id': 'try3', 'assembly_id': 'ai1', 'assembly_name': 'an1', 'extra_properties': {}}]


In [44]:
# POST to /experiment/try1/features
payload_features = {
    "page": 1,
    "page_size": 100
}
send_post_request("/experiment/try1/features", headers_post, payload_features)


POST /experiment/try1/features - Status Code: 200
Response JSON:
{'page': 1, 'page_size': 100, 'total_records': 100, 'total_pages': 1, 'features': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2', 'AADACL2-AS1', 'AADACL3', 'AADACL4', 'AADACP1', 'AADAT', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AANAT', 'AAR2', 'AARD', 'AARS1', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT', 'AASS', 'AATBC', 'AATF', 'AATK', 'ABALON', 'ABAT', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA12', 'ABCA13', 'ABCA17P', 'ABCA2', 'ABCA3', 'ABCA4', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA8', 'ABCA9', 'ABCA9-AS1', 'ABCB1', 'ABCB10', 'ABCB11', 'ABCB4', 'ABCB5', 'ABCB6', 'ABCB7', 'ABCB8', 'ABCB9', 'ABCC1', 'ABCC10', 'ABCC11', 'ABCC12', 'ABCC13', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC5', 'ABCC5-AS1', 'ABCC6', 'ABCC6P1', 'ABCC6P2', 'ABCC8', 'ABCC9', 'ABCD1', 'ABCD2', 'ABCD3', 'ABCD4', 'ABCE1', 'ABCF1', 'ABCF1-DT', 'ABCF2', 'ABCF2-H2BK1', 'ABCF3', 'ABCG1', 'ABC

In [48]:
# POST to normalization tmm
headers_normalization = {
    "accept": "application/json"
}

payload = {}

send_post_request("/normalize/try1/tmm", headers_normalization, payload)

POST /normalize/try1/tmm - Status Code: 200
Response JSON:
{'message': 'TMM normalization completed successfully'}


In [49]:
# POST to /expressions
payload_expressions = {
    "page": 1,
    "page_size": 100,
    "genes": ["ABCC5-AS1","ABCB4","AASDH","ABCB6"],
    "method": "tmm"
}
send_post_request("/expressions", headers_post, payload_expressions)

POST /expressions - Status Code: 200
Response JSON:
{'page': 1, 'page_size': 100, 'total_records': 40, 'total_pages': 1, 'query': {'page': 1, 'page_size': 100, 'genes': ['ABCC5-AS1', 'ABCB4', 'AASDH', 'ABCB6'], 'experiments': None, 'sample_ids': None, 'method': 'tmm'}, 'expressions': [{'gene_code': 'AASDH', 'sample_id': 'Control_1', 'experiment_result_id': 'try1', 'count': 131.79944855970106}, {'gene_code': 'AASDH', 'sample_id': 'Control_2', 'experiment_result_id': 'try1', 'count': 143.33068333579925}, {'gene_code': 'AASDH', 'sample_id': 'Control_3', 'experiment_result_id': 'try1', 'count': 0.0}, {'gene_code': 'AASDH', 'sample_id': 'Control_4', 'experiment_result_id': 'try1', 'count': 143.81299046800876}, {'gene_code': 'AASDH', 'sample_id': 'Control_5', 'experiment_result_id': 'try1', 'count': 144.49782309053063}, {'gene_code': 'AASDH', 'sample_id': 'Treatment_1', 'experiment_result_id': 'try1', 'count': 140.33409139029362}, {'gene_code': 'AASDH', 'sample_id': 'Treatment_2', 'experimen